In [1]:
import tensorflow as tf
from tensorflow.contrib import keras
import pandas as pd
import numpy as np

In [2]:
hidden_units = 39
input_length = 75 #length of a single sample
feature_length = 39
#batch_size = 100 #subject to change
output_size = 2

learning_rate = 0.001

In [3]:
data_path = '/Users/oosv/data/csv'

In [4]:
data_en = pd.read_csv(f"{data_path}/english_data.csv").as_matrix()

/Users/oosv/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


In [5]:
data_es = pd.read_csv(f"{data_path}/spanish_data.csv").as_matrix()

/Users/oosv/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


In [6]:
print(np.shape(data_en))
print(np.shape(data_es))

(4939049, 39)
(1760399, 39)


In [14]:
def lstm(data):
    rnn_cell = tf.nn.rnn_cell.LSTMCell(hidden_units)
    initial_state = rnn_cell.zero_state(input_length, dtype=tf.float64)
    out, state = tf.nn.dynamic_rnn(                             \
                                   cell=rnn_cell,               \
                                   inputs=data,                 \
                                   initial_state=initial_state, \
                                   dtype=tf.float64,
                                   scope='rnn2')
    
    return out

In [8]:
def dnn(data): #values yet to be decided
    model = keras.model.Sequential() 
    model.add(keras.layers.Dense(hidden_units, input_shape=(input_length,feature_length)))
    model.add(keras.layers.Activation('relu'))
    model.add(keras.layers.Dense(2))
    model.add(keyas.layers.Activation('softmax'))
    return model

In [9]:
#more setting up goes on here
def model_fn(data, labels, mode, params):
    
    output = lstm(data)

    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode, predictions=tf.argmax(output))

    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=output)
    
    # uncomment accuracy for tensorboard output
    #accuracy = tf.metrics.accuracy(labels=labels, predictions=tf.argmax(output, axis=1))
    
    optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(loss)

    if mode == tf.estimator.ModeKeys.TRAIN:
        return tf.estimator.EstimatorSpec(
            mode=tf.estomator.ModeKeys.TRAIN,
            loss=loss,
            train_op=optimizer)    
    else:
        return tf.estimator.EstimatorSpec(
            mode=tf.estomator.ModeKeys.EVAL,
            loss=loss,
            eval_metric_ops={'accuracy': \
                             tf.metrics.accuracy(labels=labels,
                                                 predictions=tf.argmax(output, axis=1))})

In [11]:
type(data_en)

numpy.ndarray

In [10]:
tensor_en = tf.convert_to_tensor(data_en[:input_length])
tensor_en = tf.reshape(tensor_en, [input_length, -1])
print(tensor_en)

Tensor("Reshape:0", shape=(100, 75, 39), dtype=float64)


In [20]:
out = lstm(tensor_en)

ValueError: Variable rnn2/lstm_cell/kernel already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "<ipython-input-14-7b46cb2aec0e>", line 5, in lstm
    scope='rnn2')
  File "<ipython-input-15-a5aba366ec64>", line 1, in <module>
    out = lstm(tensor_en)
  File "/Users/oosv/miniconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2961, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)


In [19]:
sess = tf.Session()
sess.run(out)


FailedPreconditionError: Attempting to use uninitialized value rnn2/lstm_cell/kernel
	 [[Node: rnn2/lstm_cell/kernel/read = Identity[T=DT_DOUBLE, _device="/job:localhost/replica:0/task:0/device:CPU:0"](rnn2/lstm_cell/kernel)]]

Caused by op 'rnn2/lstm_cell/kernel/read', defined at:
  File "/Users/oosv/miniconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/oosv/miniconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/oosv/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/oosv/miniconda3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/oosv/miniconda3/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 497, in start
    self.io_loop.start()
  File "/Users/oosv/miniconda3/lib/python3.6/site-packages/tornado/platform/asyncio.py", line 13